In [6]:
import numpy as np
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Input,Dropout#core layers
from tensorflow.keras.layers import Conv2D,MaxPooling2D#convolution layers
from tensorflow.keras.optimizers import Adam, RMSprop

import math
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler, Callback

In [3]:
def prepare_data():
    """
    データの前処理関数

    Returns:
    x_train(ndarray):訓練データ（50000,32,32,3）
    x_test(ndarray):テストデータ（10000,32,32,3）
    y_train(ndarray):正解ラベル（50000,10）
    y_test(ndarray):正解ラベル（10000,10）
    """
    (x_train,y_train),(x_test,y_test) = cifar10.load_data()
    #画像データを正規化
    x_train, x_test = x_train.astype('float32'),x_test.astype('float32')
    x_train, x_test = x_train/255.0, x_test/255.0
    #正解ラベルを10classのOne-Hotに変換
    y_train, y_test = to_categorical(y_train),to_categorical(y_test)

    return x_train,x_test,y_train,y_test  

In [5]:
#モデル生成
def make_convlayer():
    #畳み込み層1
    model = Sequential([
        Input(shape=(32, 32, 3)),
        Conv2D(filters=64, kernel_size=3, padding='same', activation='relu')
    ])
    #プーリング層
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #畳み込み層2
    model.add(Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'))
    #プーリング層
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #畳み込み層3
    model.add(Conv2D(filters=256, kernel_size=3, padding='same', activation='relu'))
    #プーリング層
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Flatten層
    model.add(Flatten())
    # ドロップアウト
    model.add(Dropout(0.4))
    #第7層
    model.add(Dense(512,activation='relu'))
    #出力層
    model.add(Dense(10,activation='softmax'))

    # モデルのコンパイル
    model.compile(
        optimizer=Adam(lr=0.001),
        loss=CategoricalCrossentropy(from_logits=False),
        metrics=['accuracy']
    )

    return model

In [ ]:
#ステップ減衰による学習率の推移を記録するコールバック
class LRHistory(Callback):
    def on_train_begin(self,logs={}):
        self.acc = []
        self.lr = []

    def on_epoch_end(self, batch, logs={}):
        self.acc.append(logs.get('acc'))
        self.lr.append(step_decay(len(self.acc)))

def step_decay(epoch):
    """
    ステップ減衰で学習率を降下させる関数
    Returns:学習率（float）
    """   
    initial_lrate = 0.001#初期学習率
    drop = 0.5#減衰率
    epochs_drop = 10.0#減衰を実行するエポック数
    lrate = initial_lrate * math.pow(drop,math.floor((1+epoch)/epochs_drop))
    return lrate

def train(x_train,x_test,y_train,y_test):

    model = make_convlayer()
    lr_history = LRHistory()
    lrate = LearningRateScheduler(step_decay)
    callbacks_list = [lr_history, lrate]

    #データ拡張
    datagen = ImageDataGenerator(
        width_shift_range = 0.1,#水平移動
        height_shift_range=0.1,#垂直移動
        rotation_range=10,#回転
        zoom_range=0.1#拡大
        horizontal_flip=True#左右反転
    )

    batch_size = 128
    epochs = 100

    #学習実行
    history = model.fit(
        #拡張データをミニバッチ数生成（出力は正規化）
        datagen.flow(x_train,y_train,batch_size=batch_size),
        #1回の学習におけるステップ数（画像枚数をミニバッチサイズで割った整数値）
        steps_per_epoch = x_train.shape[0] // batch_size,
        epochs = epochs,
        verbose = 1,
        validation_data = (x_test,y_test),
        callbacks = callbacks_list
    )

    return history, lr_history